# Quick intro to Semantic Memory: install, upload a doc, ask a question

This notebook shows the basic usage of Semantic Memory, via the serverless client.

First of all, install the Semantic Memory nuget, and the dotenv nuget so we can use secrets stored in a `.env` file.

In [ ]:
#r "nuget: Microsoft.SemanticMemory.Core, 0.0.230821.5-preview"

using Microsoft.SemanticMemory;

Installed Packages Microsoft.SemanticMemory.Core, 0.0.230821.5-preview

For our demo, we use also the "dotenv" nuget, to load our secret credentials from a `.env` file.
Make sure you create your `.env` file, with this content:

> ```
> OPENAI_API_KEY=<your OpenAI API key>
> ACS_ENDPOINT=<your Azure Cognitive Search endpoint>
> ACS_API_KEY=<your Azure Cognitive Search API key>
> ```

In [ ]:
#r "nuget: dotenv.net, 3.1.2"

dotenv.net.DotEnv.Load();
var env = dotenv.net.DotEnv.Read();

Now let's create an instance of Semantic Memory client using the `MemoryClientBuilder` to configure our solution. In this demo we store documents on disk, under a `tmp` folder, we use OpenAI to calculate embeddings and generate text, and Azure Cognitive Search to store embeddings and run vector search.

In [3]:
var memory = new MemoryClientBuilder()
    .WithFilesystemStorage("tmp")
    .WithOpenAIDefaults(env["OPENAI_API_KEY"])
    .WithAzureCognitiveSearch(env["ACS_ENDPOINT"], env["ACS_API_KEY"])
    .BuildServerlessClient();

At this point we have the memory client ready, we can import data and ask questions!

Let's import a PDF file with some details about Semantic Kernel:

In [4]:
await memory.ImportDocumentAsync("sample-SK-Readme.pdf", documentId: "doc001");

Now, let's ask a question about Semantic Kernel:

In [7]:
var question = "What's Semantic Kernel?";

var answer = await memory.AskAsync(question);

Console.WriteLine($"Question: {question}\n\nAnswer: {answer.Result}");

Question: What's Semantic Kernel?

Answer: Semantic Kernel is a lightweight SDK (Software Development Kit) developed by Microsoft. It enables the integration of AI Large Language Models (LLMs) with conventional programming languages. The SDK combines natural language semantic functions, traditional code native functions, and embeddings-based memory to enhance applications with AI capabilities. Semantic Kernel supports prompt templating, function chaining, vectorized memory, and intelligent planning capabilities. It encapsulates several design patterns from the latest AI research, allowing developers to infuse their applications with features like prompt chaining, summarization, zero/few-shot learning, contextual memory, embeddings, semantic indexing, planning, retrieval-augmented generation, and accessing external knowledge stores. Semantic Kernel is available for use with C# and Python programming languages. It is an open-source project, and developers are encouraged to contribute to 

And here show the sources used to generate the answer above, what we call "citations":

In [ ]:
Console.WriteLine("Sources:\n");

foreach (var x in answer.RelevantSources)
{
    Console.WriteLine($"  - {x.SourceName}  - {x.Link} [{x.Partitions.First().LastUpdate:D}]");
}

Sources:

  - sample-SK-Readme.pdf  - doc001/cf309c5bc07142dfb400a4856ef91b79 [Monday, August 21, 2023]
